# Setup

In [30]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
import statistics as stat
from IPython.display import display
import math

class par: None
class moms: None

# Load data

In [31]:
df = pd.read_csv("data/V3_timeseries__postrev_I1Y_19-Oct-2018.csv")
df = df[['year','ProfitK','ProfitY','RF', 'growthpop','priceinvt','PD','growthTFP','ik','EmpPop']]
df = df.set_index('year')
display(df.head())
display(df.describe())

,ProfitK,ProfitY,RF,growthpop,priceinvt,PD,growthTFP,ik,EmpPop
year,,,,,,,,,
1960,14.234645,29.143753,2.101667,1.646313,-1.321805,28.767187,0.647897,7.456101,56.108616
1961,14.418177,29.418518,1.855833,1.292520,-1.373270,33.813931,1.891988,7.261791,55.354179
1962,15.298585,30.143707,1.725833,1.157498,-1.162756,29.106068,3.314460,7.622490,55.513981
1963,16.091473,31.001640,2.081667,1.866188,-1.561601,30.914055,2.858746,7.922162,55.351719
1964,16.331043,31.335564,2.345833,1.675677,-0.907530,32.665569,2.761517,8.154409,55.670570


,ProfitK,ProfitY,RF,growthpop,priceinvt,PD,growthTFP,ik,EmpPop
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000
mean,14.346346,31.046356,1.414357,1.381947,-1.001414,38.333945,1.033962,7.749363,59.781546
std,1.269753,2.538903,1.995644,0.438917,1.296615,15.465343,1.618604,0.693920,2.719538
min,11.476599,27.352089,-2.533333,0.749057,-4.067451,18.496330,-3.801344,5.915066,55.351719
25%,13.559713,29.275795,-0.170000,1.022081,-1.778501,27.761936,0.119928,7.295365,57.774712
50%,14.230997,30.411072,1.451667,1.236426,-1.073029,33.334805,1.153324,7.883199,59.327110
75%,15.284401,31.860176,2.786667,1.675677,-0.097202,46.440563,2.176149,8.207948,62.339596
max,17.108320,36.831299,5.945833,2.749740,3.547138,89.555916,3.372487,9.171547,64.400581


# Moments

In [32]:
moms.ProfitK = stat.mean(df.loc[1984:2000,"ProfitK"])/100
moms.s_K = stat.mean(df.loc[1984:2000,"ProfitY"])/100
moms.ik = stat.mean(df.loc[1984:2000,"ik"])/100
moms.PD = stat.mean(df.loc[1984:2000,"PD"])
moms.TFPgrowth = stat.mean(df.loc[1984:2000,"growthTFP"])/100

for name in ['ProfitK','s_K','ik','PD','TFPgrowth']:
    print(f'{name:10s} = {getattr(moms,name):.5f}')

ProfitK    = 0.14012
s_K        = 0.29887
ik         = 0.08103
PD         = 42.33588
TFPgrowth  = 0.01100


# Step 1

Estimate $g_L$, $g_Q$ and $\bar{N}$ directly from data

In [33]:
par.g_L = stat.mean(df.loc[1984:2000,"growthpop"])/100
par.g_Q = -stat.mean(df.loc[1984:2000,"priceinvt"])/100 # note: negativ value used
par.N_bar = stat.mean(df.loc[1984:2000,"EmpPop"])/100

for name in ['g_L','g_Q','N_bar']:
    print(f'{name:5s} = {getattr(par,name):.5f}')

g_L   = 0.01171
g_Q   = 0.01769
N_bar = 0.62344


# Step 2

Estimate $\mu$, $\alpha$, $\delta$, $g_z$ by solving equation system

In [34]:
def g_T(par):
    par.g_T = (1+par.g_L)*((1+par.g_Z)**(1/(1-par.alpha)))*((1+par.g_Q)**(par.alpha/(1-par.alpha)))-1
   
def eq_footnote_15(par,moms):
     return moms.TFPgrowth -(par.g_T-(1-moms.s_K)*par.g_L-moms.s_K*(par.g_T+par.g_Q))

def eq_11(par,moms):
    return (1+par.g_L)*((1+par.g_Z)**(1/(1-par.alpha)))*((1+par.g_Q)**(par.alpha/(1-par.alpha)))-(1+par.g_T)

def eq_18(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta) - moms.ik

def eq_20(par,moms):
    return (par.mu+par.alpha-1)/(par.mu)-moms.s_K

In [35]:
def set_parameters(par,x):
    
    for name,value in zip(par.names,x):
        setattr(par,name,value)
    
    # update g_T
    g_T(par)
    
def set_x(par):
    
    x = np.zeros(len(par.names))
    for i,name in enumerate(par.names):
        x[i] = getattr(par,name)
        
    return x

def eq_system(x,par,moms):
    
    # a. set parameters
    set_parameters(par,x)            
    
    # c. evaluate equations
    out = []
    out.append(eq_footnote_15(par,moms))
    out.append(eq_11(par,moms))
    out.append(eq_18(par,moms))
    out.append(eq_20(par,moms))

    return out

In [36]:
# a. parameters to estimate
par.names = ['mu','delta','alpha','g_Z']

# b. guess
par.mu = 1.078
par.delta = 0.02778
par.alpha = 0.244
par.g_Z = 0.01298
x = set_x(par)

# c. solve
solution = optimize.fsolve(eq_system, x, args=(par,moms), full_output=0)
set_parameters(par,solution)

# d. print result
for name in par.names:
    print(f'{name:10} = {getattr(par,name):4f}')

mu         = 1.078575
delta      = 0.027777
alpha      = 0.243783
g_Z        = 0.012980


# Step 3

Estimate $\beta$ and $p$

In [55]:
# Calculate r_star
par.r_star = ((1 + par.g_T ) / moms.PD ) + par.g_T
# Find RF
moms.RF = stat.mean(df.loc[1984:2000,"RF"])/100
# Calculate g_PC
par.g_PC = (1+par.g_T)/(1 + par.g_L) - 1

Assumptions:

In [56]:
#Shock size
par.b = par.bh = 0.15
#Risk aversion coefficient
par.theta = 12
#IES, sigma = 1/IES
par.sigma = 0.5

In [57]:
# define e^(r*-rf)
exp_spread = math.exp(par.r_star - moms.RF)
# Find analytical derivation of p in appendix
par.p = (exp_spread - 1) / (((1 - par.b)**(-par.theta) + (1 + par.bh)**(-par.theta) -2) - exp_spread * ((1 - par.b)**(1-par.theta) + (1 + par.bh)**(1-par.theta) -2))

In [68]:
par.rho = par.r_star - (par.sigma * par.g_PC + par.sigma * ((1-(1/par.sigma))/(1-par.theta)) * math.log(1 + par.p * ((1 - par.b)**(1-par.theta) + (1 + par.bh)**(1-par.theta) - 2)))
par.beta = 1/(1+par.rho)

In [74]:
#Print all results
for name in ['beta','mu','p','delta','alpha','g_L', 'g_Z', 'g_Q', 'N_bar']:
    print(f'{name:10} = {getattr(par,name):4f}')

beta       = 0.960119
mu         = 1.078575
p          = 0.035884
delta      = 0.027777
alpha      = 0.243783
g_L        = 0.011706
g_Z        = 0.012980
g_Q        = 0.017692
N_bar      = 0.623442
